**Algorithm 4.2**
Import packages, set prime p and precision M. 

In [ ]:
"""Paper referenced: https://services.math.duke.edu/~dasgupta/papers/comp.pdf"""
"""TODO: May optimise by adding S where appropriate. Eg: x -> S(x)"""
import sympy,math, gmpy2
# M = 10
# p = 7
# N = 4
S = Qp(p, prec = M+math.ceil(log(M)), type = 'capped-rel', print_mode = 'series')	#Defines the padic ring of rationals, with specified precision.
R.<y> = PowerSeriesRing(S)	#Allows us to write polynomials in y with coefficients in S.  

**Integral i)**

In [ ]:
def BernPolyTilde(s, x):  
    """Returns: Tilde Bernoulli Polynomial (Pg.6)"""
    if s == 1 and round(x) == x:
        return 0
    else:
        return bernoulli_polynomial(x-math.floor(x),s)

def genDedekind(s,t,a,c):
    """Returns the generalized Dedekind sum (Pg.7)    """
    output = 0
    
    for i in range(1,c+1):
        output += BernPolyTilde(s,i/c)*BernPolyTilde(t,i*a/c)
    output = output*(c^(s-1))/(s*t)
    
    return output

def eqn14(p,a,N,c,n,k):
    """Equation 14 of the paper. prime p is fixed from start. We assumed N is prime here and the given unit is N[1]-[N]."""
    result = 0
    
    for l in range(n+1): 
        """sumofDede should iterate over all d dividng N. 
        As we assume N is prime, we only have to consider d=1 and N. Here n_N = -1 and n_1 = N"""
        sumofDede = S(-(genDedekind(k-l-1,l+1,a,c)-p^(k-l-2)*genDedekind(k-l-1,l+1,p*a,c))/(N^l)
                    +N*(genDedekind(k-l-1,l+1,a,N*c)-p^(k-l-2)*genDedekind(k-l-1,l+1,p*a,N*c)))
        result -= S(gmpy2.comb(n,l) * (a/(N*c))^(n-l) * (-1)^l) * sumofDede
        
    return result*12     

def logPowerSeries(i,p,M):
    """Returns the power series of log_p(M) around residue i, with precision M+int(log(M)).
    w is the teichmuller representative of i. If y=i(modp), log_p(y)=log_p(y/w).
    (Verified by hand, hopefully it is correct...)"""
    w = S.teichmuller(i)
    f = 0
    
    for j in range(1,M+math.ceil(log(M))):
        f -= (1/j)*(-1)^j*(y/w-1)^j
        
    return f

def gSubi35(i,p,M):  
    """Returns the function g_i(y) as in equation 35"""
    g = 1
    prec = M+math.ceil(log(M))
    
    for j in range(1,p):
        if j != i:
            g *= (y-j)^prec
            
    return g 

def hSubi35(i,p,M): 
    """Returns the power series of h_i(y)=log_p(y)/g_i(y) in eqn 35."""
    prec = M+math.ceil(log(M))
    g_i = gSubi35(i,p,M)(y+i)	#This step ensures that the power series expansion is expanded around i. 
    powerg_i = 1/(g_i+O(y^prec))
    powerg_i = powerg_i.polynomial()
    
    return powerg_i(y-i) * logPowerSeries(i,p,M)

def fEqnlogApprox35(p,M):
    """Returns the f(y) that approximates log_p(y) in eqn 35."""
    f = 0
    
    for i in range(1,p):
        f += gSubi35(i,p,M)*hSubi35(i,p,M).polynomial()
        
    return f

def integral_i(f,p,a,N,c,M):
    """Caclulates integral i) in Algo 4.2 to precision M"""
    total = 0
    exponents = f.exponents()
    coeffs = f.coefficients()
    totalLen = len(exponents)
    
    for i in range(totalLen):
        total += coeffs[i]*S(eqn14(p,a,N,c,0,exponents[i]+2)) 
        
    return total + O(p^(M+1))

def integral_i_dict(f,p,N,M):
    """Finds the integral i) for each element in the basis: [inf]-[1/N],...,[inf]-[floor(N/2)/N]."""
    S = Qp(p, prec = M+math.ceil(log(M)), type = 'capped-rel', print_mode = 'series')
    R.<y> = PowerSeriesRing(S)  
    integrals = {}
    
    for i in range(1,math.floor(N/2)+1):
        integrals[i] = integral_i(f,p,i,N,1,M)
        
    return integrals

**Integral iii)**

In [ ]:
def eqn41(p,a,N,c,n):
    """Returns the integral at Equation 41."""
    result = 0
    
    #sumofDede should iterate over all d dividng N. 
    #As we assume N is prime, we only consider d=1 and N. 
    #Here n_N = 1 and n_1 = -N
    for l in range(n+1):
        sumofDede = S(-(p^(n-l)*genDedekind(n-l+1,l+1,p*a,c)-p^n*genDedekind(n-l+1,l+1,a,c))/(N^l)
                    +N*(p^(n-l)*genDedekind(n-l+1,l+1,p*a,N*c)-p^n*genDedekind(n-l+1,l+1,a,N*c)))
        result -= S(gmpy2.comb(n,l) * (a/(N*c))^(n-l) * (-1)^l) * sumofDede
        
    return result*12     
    
def integral_iii(f,p,a,N,c,M):
    """Returns Integral iii), with inputs a/Nc, and f which is the polynomial approximation of log_p(x)"""
    total = 0
    exponents = f.exponents()
    coeffs = f.coefficients()
    totalLen = len(exponents)
    
    for i in range(totalLen):
        total += coeffs[i]*S(eqn41(p,a,N,c,exponents[i])) + O(p^(M+1))
        
    return total 

def integral_iii_dict(f,p,N,M):	
    """Finds the integral iii) for each element in the basis: [inf]-[1/N],...,[inf]-[floor(N/2)/N]."""
    integrals = {}
    
    for i in range(1,math.floor(N/2)+1):
        integrals[i] = integral_iii(f,p,i,N,1,M)
        
    return integrals 

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    f = fEqnlogApprox35(p,M).polynomial()
    integrals = integral_i_list(f,p,N,M)
    for t in integrals:
        print(t)
    integrals = integral_iii_list(f,p,N,M)
    for t in integrals:
        print(t)